<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/4.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Multi-Step Reasoning with Planning and Scratchpad</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Step 1 & 2: Summarize Templates
summarizer_a = PromptTemplate.from_template("Summarize this article:\n{article_text}")
summarizer_b = PromptTemplate.from_template("Summarize this article:\n{article_text}")

# Step 3: Compare both summaries
comparison_prompt = PromptTemplate.from_template(
    "Compare these two article summaries:\nSummary A: {summary1}\nSummary B: {summary2}\nProvide a concise comparison."
)

# Build chains
summary_a_chain = summarizer_a | llm | StrOutputParser()
summary_b_chain = summarizer_b | llm | StrOutputParser()

# Compose workflow
# We use lambda functions to extract the nested 'article_text' for each specific chain
workflow = {
    "summary1": lambda x: summary_a_chain.invoke(x["summary1"]),
    "summary2": lambda x: summary_b_chain.invoke(x["summary2"]),
} | comparison_prompt | llm | StrOutputParser()

# Example input
inputs = {
    "summary1": {"article_text": "Article A discusses AI policy changes in Europe."},
    "summary2": {"article_text": "Article B analyzes new regulations on AI safety in Asia."}
}

result = workflow.invoke(inputs)
print(result)